In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-39.4"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2022, 3)
val monthlyReportForecastUntil = YearMonthTO(2022, 9)

val yearlyReportFrom = 2019
val yearlyReportTo = 2021
val yearlyReportForecastUntil = 2023

val workFundName = "Work Income"
val workReportViewName = "Work income report"
val WORK_REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/work-report-data-configuration.yaml"

val savingsFundName = "Savings"
val savingsReportViewName = "Savings report"
val SAVINGS_REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/savings-report-data-configuration.yaml"

val otherIncomeFundName = "Other Income"
val otherIncomeReportViewName = "Other income report"
val OTHER_INCOME_REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/other-income-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = client.ensureUserExists(username)
user

UserTO(id=c05bfaa2-e9ba-459b-8173-bf8aed20c8ef, username=Johann-39.4)

In [3]:
val workReportView = client.createReportView(user, workReportViewName, workFundName, client.fromYaml(File(WORK_REPORT_DATA_CONFIGURATION_YAML_FILE), "dataConfiguration"))
workReportView

ReportViewTO(id=926d4b18-7033-4bfa-9fa2-0f32af755fc5, name=Work income report, fundId=4dd079f2-3d35-4b17-a00d-7f33be64ea6e, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), groups=[ReportGroupTO(name=income, filter=RecordFilterTO(labels=[income])), ReportGroupTO(name=taxes, filter=RecordFilterTO(labels=[work_taxes]))], reports=ReportsConfigurationTO(net=NetReportConfigurationTO(enabled=false, filter=null), valueReport=ValueReportConfigurationTO(enabled=true, filter=null), groupedNet=GenericReportConfigurationTO(enabled=true), groupedBudget=GroupedBudgetReportConfigurationTO(enabled=false, distributions=[])), forecast=ForecastConfigurationTO(inputBuckets=1)))

In [4]:
val monthlyWorkReportData = client.getMonthlyReportViewData(user, workReportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [5]:
val yearlyWorkReportData = client.getYearlyReportViewData(user, workReportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [6]:
val savingsReportView = client.createReportView(user, savingsReportViewName, savingsFundName, client.fromYaml(File(SAVINGS_REPORT_DATA_CONFIGURATION_YAML_FILE)))

In [7]:
val monthlySavingsReportData = client.getMonthlyReportViewData(user, savingsReportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [8]:
val otherIncomeReportView = client.createReportView(user, otherIncomeReportViewName, otherIncomeFundName, client.fromYaml(File(OTHER_INCOME_REPORT_DATA_CONFIGURATION_YAML_FILE)))

In [9]:
val monthlyOtherIncomeReportData = client.getMonthlyReportViewData(user, otherIncomeReportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)

In [10]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotWorkGroupedNetData(reportData: ReportDataTO, plotTitle: String, incomeGroupName: String, taxesGroup: String): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "income" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == incomeGroupName }?.net ?: BigDecimal.ZERO
        },
        "taxes" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == taxesGroup }?.net ?: BigDecimal.ZERO
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedNet!!.mapNotNull { it.net }.fold(BigDecimal.ZERO) { acc, value -> acc + value }
        },
        "value" to reportData.data.map { bucket ->
            bucket.value!!.end ?: BigDecimal.ZERO
        }
    )

    return dataFrame
        .plot {
            line {
                val values =
                    listOf("net", "taxes", "value").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("net")
                color = Color.YELLOW
            }
            line {
                y("taxes")
                color = Color.RED
            }
            line {
                y("income")
                color = Color.GREEN
            }
            area {
                y("value")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}

fun plotSingleDimensionGroupedNetData(reportData: ReportDataTO, plotTitle: String, profitGroupName: String): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "profit" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == profitGroupName }?.net ?: BigDecimal.ZERO
        },
    )

    return dataFrame
        .plot {
            line {
                val values =
                    listOf("profit").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("profit")
                color = Color.YELLOW
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}


In [11]:
yearlyWorkReportData

ReportDataTO(viewId=926d4b18-7033-4bfa-9fa2-0f32af755fc5, interval=ReportDataIntervalTO(granularity=YEARLY, fromDate=2019-01-01, toDate=2021-12-31, forecastUntilDate=2023-12-31), data=[ReportDataItemTO(timeBucket=DateIntervalTO(from=2019-01-01, to=2019-12-31), bucketType=REAL, net=null, value=ValueReportTO(start=0.0, end=0.0, min=0.0, max=0.0), groupedNet=[ReportDataGroupedNetItemTO(group=income, net=107567.0), ReportDataGroupedNetItemTO(group=taxes, net=0.0)], groupedBudget=null), ReportDataItemTO(timeBucket=DateIntervalTO(from=2020-01-01, to=2020-12-31), bucketType=REAL, net=null, value=ValueReportTO(start=0.0, end=0.0, min=0.0, max=0.0), groupedNet=[ReportDataGroupedNetItemTO(group=income, net=138677.75), ReportDataGroupedNetItemTO(group=taxes, net=0.0)], groupedBudget=null), ReportDataItemTO(timeBucket=DateIntervalTO(from=2021-01-01, to=2021-12-31), bucketType=REAL, net=null, value=ValueReportTO(start=0.0, end=46986.81, min=0.0, max=0.0), groupedNet=[ReportDataGroupedNetItemTO(grou

In [12]:
plotWorkGroupedNetData(yearlyWorkReportData, "Work income per year", "income", "taxes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="m9EJRa"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Work income per year"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"income":[107567.0,138677.75,213978.33,153407.6933333333,153407.6933333333],
"taxes":[0.0,0.0,-10510.52,-3503.506666666667,-3503.506666666667],
"net":[107567.0,138677.75,203467.81,149904.18666666665,149904.18666666665],
"value":[0.0,0.0,46986.81,62649.08,78311.35]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[-10510.52,203467.81]
},
"sampling":"none",
"x":1.6250976E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"net"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"taxes"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"income"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"value"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("m9EJRa");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 
 
 -20,000 
 
 
 
 
 
 
 -10,000 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 

In [13]:
plotWorkGroupedNetData(monthlyWorkReportData, "Work Income per month", "income", "taxes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="w7nxxn"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Work Income per month"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"income":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,8344.0,8007.0,10203.0,10287.0,10381.0,10176.0,10237.0,10229.0,19367.0,11411.0,10040.0,10082.0,10489.0,10110.0,10124.0,11694.0,14718.75,13596.0,13500.0,13595.0,13836.0,14227.0,20886.4,19842.08,18797.76,14620.48,23497.2,24103.77,23476.64,23609.78,14915.04,17416.96,19102.42583333333,19541.29465277778,19984.15254050926,19908.96525221836,19914.53902323656,20007.60394183961],
"taxes":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-500.0,0.0,0.0,-344.81,-1583.32,-2281.51,-345.0,-346.0,-4763.44,-346.44,-346.44,-5786.46,-347.0,-1374.201666666667,-1488.718472222222,-1612.778344907407,-1718.442373649691,-1729.702571453832,-1683.718619074985],
"net":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,8344.0,8007.0,10203.0,10287.0,10381.0,10176.0,10237.0,10229.0,19367.0,11411.0,10040.0,10082.0,10489.0,10110.0,10124.0,11694.0,14718.75,13596.0,13500.0,13095.0,13836.0,14227.0,20541.59,18258.76,16516.25,14275.48,23151.2,19340.33,23130.2,23263.34,9128.58,17069.96,17728.224166666663,18052.57618055556,18371.374195601853,18190.52287856867,18184.83645178273,18323.885322764625],
"value":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20541.59,2036.35,18552.6,32828.08,4516.28,23856.61,46986.81,70250.15,14707.73,31777.69,34425.83083333333,37294.65006944445,38690.738408564815,41745.27077594522,43677.99334060732,44582.15278565793]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.646092

In [14]:
plotSingleDimensionGroupedNetData(monthlySavingsReportData, "Savings income per month", "profit")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="WcVzcb"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Savings income per month"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"profit":[0.03,1.68,3.63,5.44,8.19,3.79,4.39,2.81,1.1,0.94,5.32,6.53,8.14,12.55,17.01,28.52,30.82,26.59,18.98,8.71,0.62,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.97,3.4,20.74,2.5925,2.808541666666667,3.042586805555556,3.296135706018519,3.570813681520062,3.8683814883134]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[0.0,30.82]
},
"sampling":"none",
"x":1.6473888E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"profit"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"8"
};
 var containerDiv = document.getElementById("WcVzcb");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

In [15]:
plotSingleDimensionGroupedNetData(monthlyOtherIncomeReportData, "Other income per month", "income")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Wp8hrd"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Other income per month"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"profit":[0.0,0.0,500.0,150.0,0.0,746.0,0.0,1500.0,1500.0,1250.0,1030.0,1500.0,750.0,750.0,23380.0,0.0,200.0,0.0,0.0,7743.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4750.0,5145.833333333333,5574.652777777778,6039.207175925926,6542.47444058642,2337.680643968621]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12,1.6566336E12,1.659312E12,1.6619904E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[0.0,57000.0]
},
"sampling":"none",
"x":1.6473888E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"profit"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"11"
};
 var containerDiv = document.getElementById("Wp8hrd");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
